In [17]:
import numpy as np
import pandas as pd
from carbon_models import *
pd.options.plotting.backend = "plotly"

In [18]:
# Import data
df_carbon = pd.read_csv("carbon_data.csv",sep=",")
df_carbon.columns

Index(['Date', 'Region', 'saison', 'week_day', 'is_holiday', 'is_bank_holiday',
       'Temp', 'Temp1', 'Temp2', 'Temp3', 'Temp4', 'Temp5', 'Temp6', 'Temp7',
       'ce', 'ce1', 'ce2', 'ce3', 'ce4', 'ce5', 'ce6', 'ce7', 'te', 're',
       'Emissions'],
      dtype='object')

In [19]:
# Define parameters
train_start = '2019-01-01'
train_end = '2019-10-01'
test_end = '2019-12-31'

best_vars = [
    "Region",
    "Temp",
    "saison",
    "week_day",
    "is_holiday",
    "is_bank_holiday",
    "ce",
    "te",
    "re",
]

In [20]:
# Train the model and predict
# output_df = LSTM_predict(df_carbon,train_start,train_end,test_end)
# output_df = Catboost_predict(df_carbon,train_start,train_end,test_end,best_vars)
output_df = Xgboost_predict(df_carbon,train_start,train_end,test_end,best_vars)

In [21]:
output_df

,Date,Region,y,y_pred
0,2019-01-01,Auvergne-Rhône-Alpes,-0.085974,NaN
1,2019-01-01,Bourgogne-Franche-Comté,-0.968110,NaN
2,2019-01-01,Bretagne,-0.814845,NaN
3,2019-01-01,Centre-Val de Loire,-1.017309,NaN
4,2019-01-01,Grand Est,-0.538966,NaN
...,...,...,...,...
4375,2019-12-31,Nouvelle-Aquitaine,-0.012654,0.596745
4376,2019-12-31,Occitanie,-0.061643,0.230987
4377,2019-12-31,Pays de la Loire,-0.553571,-0.328904
4378,2019-12-31,Provence-Alpes-Côte d'Azur,-0.152140,0.189823


In [10]:
# Plot for the whole country
output_df.groupby(['Date']).apply(pd.DataFrame.sum,skipna=False).drop(columns=['Date','Region']).plot(title='France')

In [11]:
# Plot for the region `reg`
REGIONS = np.unique(df_carbon['Region'])
reg = REGIONS[8]

output_df[output_df['Region'] == reg].drop(columns='Region').set_index('Date').plot(title=reg)

In [14]:
is_test = (output_df['Date'] > train_end) * (output_df['Date'] <= test_end)

y_test = output_df['y'][is_test]
y_pred = output_df['y_pred'][is_test]

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Evaluate using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")

# Evaluate using Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Evaluate using Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae}")

# Evaluate using Coefficient of Determination (R^2)
r2 = r2_score(y_test, y_pred)
print(f"Coefficient of Determination (R^2): {r2}")

Mean Squared Error (MSE): 0.23210387146709702
Root Mean Squared Error (RMSE): 0.48177159678326514
Mean Absolute Error (MAE): 0.3600133276328732
Coefficient of Determination (R^2): 0.8008904343174896
